In [27]:
import sys
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import make_column_transformer

sys.path.append("/Users/nwong/Workspace/Projects/tdt4173_project/src")

from feature_engineering.sklearn_transformers import *
from feature_engineering.store_features import *
from feature_engineering.bus_stop_features import *
from feature_engineering.utils import *
import h2o
from h2o.automl import H2OAutoML

stop_importance_levels = ['Mangler viktighetsnivå',
                          'Standard holdeplass',
                          'Lokalt knutepunkt',
                          'Nasjonalt knutepunkt',
                          'Regionalt knutepunkt',
                          'Annen viktig holdeplass']
store_types = ['lv1_desc', 'lv2_desc', 'lv3_desc', 'lv4_desc']

In [28]:
stores_train_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_train.csv"))
stores_test_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_test.csv"))
stores_extra_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_extra.csv"))

income_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_income_households.csv"))
age_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_age_distribution.csv"))
household_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_households_num_persons.csv"))
grunnkrets_df = combine_keys(set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_norway_stripped.csv")))
plaace_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/plaace_hierarchy.csv")

bus_stops_df = bus_stops_lat_lon(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/busstops_norway.csv"))


/Users/nwong/Workspace/Projects/tdt4173_project/src/feature_engineering/bus_stop_features.py:11: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  bus_stops_df[['lon', 'lat']] = bus_stops_df['lng_lat'].str.split(


In [29]:
stores_train_merged = stores_train_df.merge(plaace_df, on="plaace_hierarchy_id", how="left").drop(['revenue'], axis=1)
stores_test_merged = stores_test_df.merge(plaace_df, on="plaace_hierarchy_id", how="left")
stores_extra_merged = stores_extra_df.merge(plaace_df, on="plaace_hierarchy_id", how="left")

In [31]:
merged_stores_train = stores_train_df \
    .merge(income_dist, on="grunnkrets_id", how="left") \
    .merge(bus_stops_distance_by_importance(stores_train_df, bus_stops_df, stop_importance_levels).reset_index(level=0), on="store_id", how="left") \
    .merge(store_closest_by_store_groups(stores_train_merged, pd.concat([stores_train_merged, stores_test_merged, stores_extra_merged]), store_types), on="store_id", how="left")
merged_stores_test = stores_test_df \
    .merge(income_dist, on="grunnkrets_id", how="left") \
    .merge(bus_stops_distance_by_importance(stores_test_df, bus_stops_df, stop_importance_levels).reset_index(level=0), on="store_id", how="left") \
    .merge(store_closest_by_store_groups(stores_test_merged, pd.concat([stores_train_merged, stores_test_merged, stores_extra_merged]), store_types), on="store_id", how="left")

In [32]:
merged_stores_train

,store_id,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,...,distance_to_mangler_viktighetsnivå,distance_to_standard_holdeplass,distance_to_lokalt_knutepunkt,distance_to_nasjonalt_knutepunkt,distance_to_regionalt_knutepunkt,distance_to_annen_viktig_holdeplass,distance_to_lv1_desc,distance_to_lv2_desc,distance_to_lv3_desc,distance_to_lv4_desc
0,983540538-974187930-44774,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,...,0.005315,0.012600,0.005467,0.294614,0.016952,0.155313,6.266641e-04,6.266641e-04,4.695977e-02,0.046960
1,987074191-973117734-44755,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,...,0.001579,0.045599,0.001257,0.020375,0.001559,0.116319,3.596392e-07,3.151592e-04,1.864898e-03,0.001865
2,984890265-981157303-64491,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,...,0.000371,0.011161,0.025783,0.326362,0.017330,0.054763,8.339069e-04,8.339069e-04,3.571459e-01,0.357146
3,914057442-992924179-126912,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,...,0.005329,0.010858,0.016854,0.012352,0.000522,5.257526,3.480443e-04,3.480443e-04,4.327842e-02,0.043278
4,913018583-913063538-668469,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,...,0.003096,0.002093,0.003370,0.319223,0.644539,0.006737,3.908446e-07,1.208005e-03,2.808519e-03,0.002809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12854,915789943-915806929-781991,MEIERIGÅRDEN BRYGGERIUTSALG,2.8.11.2,Beer and soda shop,7010705,THUEGATA 2,59.416276,10.480970,NaN,NaN,...,0.001349,0.008576,0.006203,0.176835,0.005978,0.009062,2.546594e-04,1.679672e-03,5.398634e-03,0.010215
12855,917921733-917982368-868081,GULATING ØLUTSALG CC GJØVIK,2.8.11.2,Beer and soda shop,5020406,NaN,60.799991,10.693635,GULATING GRUPPEN,CC Gjøvik,...,0.002951,0.118968,0.022918,0.333546,0.051408,0.168095,3.666254e-07,3.666254e-07,3.666254e-07,0.259925
12856,911721961-911764474-496764,GULATING ØLUTSALG STRØMMEN,2.8.11.2,Beer and soda shop,2310803,STØPERIVEIEN 6,59.946562,11.007659,GULATING GRUPPEN,Strømmen Storsenter,...,0.001587,0.102015,0.025107,0.249433,0.032679,0.037453,5.596648e-04,5.596648e-04,5.596648e-04,0.059335
12857,914337046-914343372-721294,DET GODE BRYGG,2.8.11.2,Beer and soda shop,11020113,VÅGSGATA 16,58.850261,5.735674,NaN,Bystasjonen,...,0.001097,0.005468,0.080072,0.116255,0.144443,0.101411,3.492682e-04,4.838586e-04,3.165082e-03,0.126748


In [33]:
pt = PowerTransformer()
rev_transformed = pt.fit_transform(merged_stores_train[["revenue"]])
merged_stores_train["revenue"] = rev_transformed

In [34]:
# Start the H2O cluster (locally)
h2o.init()

# Import a sample binary outcome train/test set into H2O
train = h2o.H2OFrame(merged_stores_train)
test = h2o.H2OFrame(merged_stores_test)
#test = h2o.H2OFrame(pd.concat([test_set, stores_test_enriched[['store_id']]], axis=1).drop(drop_cols, axis=1))

# Identify predictors and response
x = train.columns
y = "revenue"
x.remove(y)

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 20 mins
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,"14 days, 2 hours and 30 minutes"
H2O_cluster_name:,H2O_from_python_nwong_224gip
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.070 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
18:26:08.913: _train param, Dropping bad and constant columns: [store_id, store_name]

███
18:26:22.696: _train param, Dropping bad and constant columns: [store_id, store_name]

██
18:26:45.515: _train param, Dropping bad and constant columns: [store_id, store_name]


18:26:49.687: _train param, Dropping bad and constant columns: [store_id, store_name]

█
18:27:00.632: _train param, Dropping bad and constant columns: [store_id, store_name]

██
18:28:04.624: _train param, Dropping bad and constant columns: [store_id, store_name]
18:28:06.966: _train param, Dropping bad and constant columns: [store_id, store_name]

█
18:28:09.754: _train param, Dropping bad and constant columns: [store_id, store_name]

█
18:28:13.860: _train param, Dropping bad and constant columns: [store_id, store

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
# The leader model is stored here
aml.leader

In [ ]:
m = h2o.get_model(lb[3,"model_id"])
m.varimp(use_pandas=True)

In [ ]:
preds_avg = aml.predict(test)
preds_best = aml.leader.predict(test)

In [ ]:
df = test.cbind(preds_best)
df = df.as_data_frame(use_pandas=True)
result = df.loc[:,("store_id", 'predict')]
submission = result.rename(columns = {"store_id": "id",  "predict" : "predicted"})
submission['predict'] = pt.inverse_transform(submission[['predict']])
submission.to_csv("StackedEnsembleBestOfFamily2.csv", index = False)

submission